In [13]:
import networkx as nx
import random
import copy
import csv


In [14]:
# file 1 = CA-AstroPh.csv
# file 2 = ca-GrQc.csv
# file 3 = CA-HepTh.csv
path= 'CA-HepTh.csv'
file = open(path)



In [15]:
#took help from stackoverflow

graph, nodes = [], []

with open(path, newline="") as csv_file:
    reader = csv.reader(csv_file)
    # Discard the header (if there is one)
    next(reader)
    for line in reader:
        source, target = [int(item) for item in line]
        nodes.append(source)
        nodes.append(target)
        # Have to append a tuple if you want them together.
        graph.append((source, target))
    
nodes = list(set(nodes))
n = 50

In [16]:

degree = {}

for edge in graph: 
    if edge[0] in degree:
        degree[edge[0]] += 1
    else:
        degree[edge[0]] = 1
    if edge[1] in degree:
        degree[edge[1]] += 1
    else:
        degree[edge[1]] = 1

seeds = list({k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)}.keys())[:n]

In [17]:
def independent_cascade(DG, seeds):

    A = copy.deepcopy(seeds)  # prevent side effect
    return _diffuse_k_rounds(DG, A,1)


In [18]:

def _diffuse_k_rounds(G, A, steps):
    tried_edges = set()
    layer_i_nodes = [ ]
    layer_i_nodes.append([i for i in A])
    while steps > 0 and len(A) < G.number_of_nodes():
        len_old = len(A)
        (A, activated_nodes_of_this_round, cur_tried_edges) = _diffuse_one_round(G, A, tried_edges)
        layer_i_nodes = layer_i_nodes[0] + activated_nodes_of_this_round
        tried_edges = tried_edges.union(cur_tried_edges)
        if len(A) == len_old:
            break
        steps -= 1
    
    return layer_i_nodes

In [19]:
def _diffuse_one_round(G, A, tried_edges):
    activated_nodes_of_this_round = []
    cur_tried_edges = []
    for s in A:
        for nb in G.neighbors(s):
            if nb in A or (s, nb) in tried_edges or (s, nb) in cur_tried_edges:
                continue
            if (_prop_success(G, s, nb)and (s,nb) not in cur_tried_edges):
                activated_nodes_of_this_round.append(nb)
                cur_tried_edges.append((s, nb))
                activated_nodes_of_this_round = list(activated_nodes_of_this_round)
                A.extend(activated_nodes_of_this_round)
        anots=set(activated_nodes_of_this_round)
        cte=set(cur_tried_edges)
        return A,list(anots) ,list(cte) 




In [20]:
def _prop_success(G, src, dest):
    return random.random() <= G[src][dest]['weight']


In [21]:

edgelist = path #the input graph file with edge weights
cascadefilename = 'op' #name of the file for storing output cascades

In [22]:
G=nx.DiGraph() #generate graph from edge lis

with open(path, newline="") as csv_file:
    reader = csv.reader(csv_file)
    # Discard the header (if there is one)
    next(reader)
    for line in reader:
        source, target = [int(item) for item in line]
        
        G.add_edge(source, target,weight = 0 )
        G[source][target]['weight'] = G.in_degree(target)
    
n = nx.number_of_nodes(G)
print(n)

9877


In [23]:
cascades = independent_cascade(G, seeds )

In [24]:

print(cascades)#using this to find the nodes infected


[1441, 19615, 63113, 30744, 16164, 59077, 23420, 48973, 44262, 13648, 61742, 40517, 30160, 20394, 62227, 33512, 54785, 66135, 39085, 48192, 27587, 43226, 48570, 6142, 60926, 11403, 33715, 24394, 36860, 46139, 29595, 17370, 29715, 50568, 17289, 5138, 14017, 57878, 59471, 55319, 7859, 68111, 54915, 14726, 65168, 63697, 14642, 41687, 45385, 44515, 36481, 1539, 5510, 38025, 28042, 53387, 59789, 19342, 27024, 31760, 40852, 11925, 26645, 28439, 35096, 23962, 53406, 22559, 36767, 26913, 42018, 43425, 54691, 31525, 9638, 31398, 51240, 53159, 54696, 65568, 65325, 67757, 44721, 8502, 25149, 702, 47293, 49092, 18120, 43080, 2765, 5581, 10574, 23119, 19665, 21073, 26446, 27218, 22743, 65114, 61147, 57820, 67173, 2410, 63979, 6638, 10863, 39920, 15089, 14068, 15604, 22264, 24571, 7806]


In [25]:
no_of_nodes_infected=len(cascades)
print(no_of_nodes_infected)

114


In [26]:
print((no_of_nodes_infected/n)*100) # percentage infection

1.1541966184063988
